In [150]:
import pandas as pd
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

Информация по талицам:

contractor_test - информация о контрагентах 

 - ITN - ИНН
 - name_contractor - название компании контрагента
 - form - передача форм собственности предприятий

 
contractor_test - информация о контранках

 - number - номер договора
 - date - дата заключения договора
 - ITN_contractor - ИНН контрагента, с которым заключен Договор
 - our_template	 - наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)
 - price - стоимость контракта 
 - object - объект строительства
 - gu - гарантийные удержания
 - gp - ген.услуги
 - prepayment - аванс (Доля от стоисоти договора)


In [151]:
# Информация о Контрактах

class WorkWithContract:
    data : pd.DataFrame
    copy: bool
    
    """# Нужно доработать тонкости связанные с типами вводимых данных"""
    
    def __init__(self, data, copy: bool = False):
        self.data = data
        self.info = {'number' : 'номер договора',
                'date': 'дата заключения договора в формате DD.MM.YYYY',
                'ITN_contractor' : 'ИНН контрагента, с которым заключен Договор',
                'price' : 'стоимость контракта', 
                'object' : 'объект строительства', 
                'our_template': 'наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)',
                'gu' : 'гарантийные удержания',
                'gp' : 'ген.услуги',
                'prepayment' : 'аванс (Доля от стоисоти договора)'}
        self.gu = 0.05
        self.gp = 0.02
        self.prepayment = 0.30
        if copy:
            self.data = data.copy()
        else:
            self.data = data
            
    
        
    def check_number(self, other):
# Проверка на наличие номера Договора в Базе
        if self.data.isin({'number' : other}).any()[0]:
            print(f'Контракт c № {other} уже есть в Базе')
        else:
            print(f'Номер {other} свободен, можно использовать!')
            
            
    def our_template(self):
        pass
        
        
    def new_contract(self):      
# Добавление нового контракта в Базу
        new_row = {}
        for i in self.data.columns:
            print(f'Введите информацию {self.info[i]}')
            a = input()
            if i == 'date':
                a = datetime.strptime(a, '%d.%m.%Y')
            elif i in ['price', 'gu', 'gp', 'prepayment']:
                a = float(a)
            new_row[i] = a
            if i == 'our_template':
                if a == '1':
                    new_row['gu'] = self.gu
                    new_row['gp'] = self.gp
                    new_row['prepayment'] = self.prepayment
                    print('Размер Гарантийных удержаний и Ген услуг проставлены согласно Стандартного шаблона Договора')
                    break            
        new_row_1 = pd.Series(data=new_row)
        self.data = self.data.append(new_row_1, ignore_index=True)
        print(f'Договор записан')
        return self.data
    
# Удаление Договора из базы по номеру Договора
# Можно переносить его в отдельную Базу

    def drop_contract(self, other):     
        self.data.set_index('number', inplace = True)
        self.data.drop(other, inplace = True)
        print(f'Договор № {other} удален из базы!!!!')
        self.data.reset_index(inplace=True)
        return self.data
    
    def change_terms(self):
# Внесение изменений 
        pass

    
    def end_work(self):
# Конец работы с документом и запись новой информации
        today_day = datetime.today().strftime('%d-%m-%Y')
        name_file = 'contract_{}.csv'
        self.data.to_csv(name_file.format(today_day), index = False)
        print('Файл записан')
        





In [152]:
data = pd.read_csv('contract_02-06-2022.csv')

In [153]:
new = WorkWithContract(data = data)
# new.new_contract()

In [146]:
type(new.data['date'].loc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [147]:
new.data

,number,date,INT_contractor,price,object,our_template,gu,gp,prepayment
0,ЛТС-123,2021-12-02,7816659721,1300000.0,NEW_TIME,1,0.05,"0,02",0.0
1,234,2022-12-06,1657147001,14000000.0,NEW_TIME,0,0.00,0,1.0
2,LTS-156,2019-12-08,1111111111,13000.0,NEW_TIME,0,0.00,0.05,0.1


In [148]:
new.end_work()

Файл записан


In [212]:
# Информация о Контрагентах

class WorkWithContractor():
    data : pd.DataFrame
    
    def __init__(self, data):
        self.data = data #База со сведеньями о Контрагентах
        self.info = {'ITN': 'ИНН контрагента',
                'name_contractor':'название компании контрагента',
                'form' : 'передача форм собственности предприятий'}
    
    def check_contractor(self):
# Проверка на наличие контрагента в Базе по ИНН
        self.data['ITN'] = self.data['ITN'].astype(str) # Убрать после того, как разберусь с типами во всех таблица
        print(f'Введите ИНН контрагента')
        ITN_contractor = input()
        if self.data.isin({'ITN' : [ITN_contractor]}).any()[0]:
            print(f'Контрагент с ИНН {ITN_contractor} уже есть в Базе')
            return self.data[self.data['ITN'] == ITN_contractor]
        else:
            print(f'Контрагентa c ИНН {ITN_contractor} нет в Базе Контрагентов')
            
    def new_contractor(self):      
# Добавление нового контракта в Базу
        new_row = {}
        for i in self.data.columns:
            print(f'Введите информацию {self.info[i]}')
            a = input()
            new_row[i] = a     
        new_row_1 = pd.Series(data=new_row)
        self.data = self.data.append(new_row_1, ignore_index=True)
        print(f'Контрагент записан в Базу')
        return self.data
    
    def drop_contractor(self, other):     
        self.data.set_index('ITN', inplace = True)
        self.data.drop(other, inplace = True)
        print(f'Контрагент {other} удален из базы!!!!')
        self.data.reset_index(inplace=True)
        return self.data
            
    def end_work(self):
# Конец работы с документом и запись новой информации
        today_day = datetime.today().strftime('%d-%m-%Y')
        name_file = 'contractor_{}.csv'
        self.data.to_csv(name_file.format(today_day), index = False)
        print('Файл записан')

In [213]:
cont = pd.read_csv('contractor_02-06-2022.csv')

In [214]:
contractor = WorkWithContractor(cont)

In [215]:
contractor.data

,ITN,name_contractor,form


In [223]:
contractor.new_contractor()

Введите информацию ИНН контрагента
7811583941
Введите информацию название компании контрагента
СК ПЕГАС
Введите информацию передача форм собственности предприятий
ООО
Контрагент записан в Базу


,ITN,name_contractor,form
0,7810727284,АКВАПОИНТ,ООО
1,7840516562,АРСЕНАЛ СПБ,ООО
2,7810299920,АРТ-КЛИМАТ,ООО
3,7810582416,ВОЛЬТ СПБ,ООО
4,7813539105,ЗАЛЕЙ ПОЛ,ООО
5,7839409453,ИК АЛЬЯНС,ООО
6,7811583941,СК ПЕГАС,ООО


In [224]:
contractor.end_work()

Файл записан


In [13]:
# Расчёт задолности по Объекту
class ObjectDebt(): 
    
    def __init__(self, data):
        self.data = data
    
# Расчёт задолности по Контрагенту


class CotractorDebt():

    def __init__(self, data):
        self.data = data

# Расчёт задолности по Контракту

class ContractDebt():
    pass

In [3]:
contractor = pd.read_csv('contractor_02-06-2022.csv.csv')

In [268]:
contractor.iloc[0]['adress']

,ITN,name_contractor,form
0,7816659721,ПСК Аракс,ООО
1,7811147430,ПЕТРОПРОФИЛЬ ПЛЮС,OOO
2,1657147001,ОТИС ГРУПП,'OOO'


In [269]:
contractor[contractor['ITN'] == 7816659721]

,ITN,name_contractor,form
0,7816659721,ПСК Аракс,ООО


In [276]:
contractor.isin({'ITN' : [7816659721]}).any()[0]

True

In [191]:
data2 = pd.read_csv('contract _test.csv', sep =';')

In [192]:
data2

,number,data,INT_contractor,our_template,price,object,gu,gp,prepayment
0,ЛТС-123,12.02.2021,7816659721,1,1300000.0,NEW_TIME,0.05,"0,02",0


In [245]:
def check_number1(data, number_contr):
    if data.isin({'number' : [number_contr]}).any()[0]:
        print(f'Контракт c № {number_contr} уже есть в Базе')
    else:
        print(f'Номер {number_contr} свободен, можно использовать!')

In [ ]:
def new_contract1(data):
    new_row = {}
    for i in data.columns:
        print(f'Введите информацию {i}')
        a = input()
        new_row[i] = a
#             if i == 'our_template':
#                 if a == 1:
    new_row = pd.Series(data=new_row)
    return data.append(new_row, ignore_index=True)

In [194]:
def drop_contract1(data, number_contr):
        """ Удаление по номеру контракта"""
        data.set_index('number', inplace = True)
        data.drop(number_contr, inplace = True)
        print(f'Договор № {number_contr} удален из базы')
        data.reset_index(inplace=True)
        return data

In [ ]:
def check_contractor1(contractor):
# Проверка на наличие контрагента в Базе по ИНН
    contractor['ITN'] = contractor['ITN'].astype(str) # Убрать после того, как разберусь с типами во всех таблица
    print(f'Введите ИНН контрагента')
    ITN_contractor = input()
    if contractor.isin({'ITN' : [ITN_contractor]}).any()[0]:
        print(f'Контрагент с ИНН {ITN_contractor} уже есть в Базе')
        return contractor[contractor['ITN'] == ITN_contractor]
    else:
        print(f'Контрагентa c ИНН {ITN_contractor} нет в Базе Контрагентов')

In [8]:
today_day = datetime.today().strftime('%d-%m-%Y')

In [10]:
name_file = 'money_{}.csv'

In [11]:
name_file.format(today_day)

'money_02-06-2022.csv'